# Box2D Car Racing 분석 방법

Box2D Car Racing은 앞서 설명한대로 OpenAIGym에서 제공하는 오픈 소스 게임이다. 해당 게임에 대한 자세한 설명은 [문서](https://www.gymlibrary.dev/environments/box2d/car_racing/)에서 확인할 수 있다. 아래에는 프로젝트 이해, 활용에 필요한 내용만을 정리하여 작성한다.

## Car Racing 환경

<img src = "https://www.gymlibrary.dev/_images/car_racing.gif" height = 150 width = 200>

게임이 시작되면 자동차는 사용자의 입력을 받아 움직이며, 도로를 따라 경로를 완주하는 것을 목표로 한다. 트랙의 모양은 랜덤으로 생성된다.

화면 아래 표시된 정보는 순서대로
1. 속도
2. 4개의 ABS 센서
3. 바퀴의 회전 방향
4. 자이로스코프

를 나타낸다. 자세한 설명은 뒤에 다시 언급한다.

## 1. 기본 정보
- **게임 화면 크기** : (96,96, 3)
- **액션(Action)**
  
  1. continous인 경우 총 3가지의 액션이 있다.
    * 회전(오른쪽, 왼쪽)
    * Gas (전진, 가속)
    * Brake
  2. discrete인 경우, 총 5가지의 액션이 있다.
    * 아무것도 안하기
    * 왼쪽 회전
    * 오른쪽 회전
    * Gas (전진, 가속)
    * Brake
  
  뒷바퀴를 돌려서 주행하는 방식인 후륜구동 방식으로 진행된다. 따라서 악셀과 회전을 동시에 입력하는 것을 금한다.

- **보상(Reward)**

  1. 매 프레임 당 -0.1 씩 제공
  2. 매 트랙이 방문될때마다 +1000/N (N은 방문해야하는 총 트랙의 수)

- **게임 종료**
  1. 한 에피소드는 agent가 처음 시작위치로 돌아왔을 때, 모든 타일의 `lap_complete_percent`이상 방문되었으면 종료된다.
  2. 차가 playfield를 벗어나면 보상으로 -100을 받고 사망한다.



## 2. 코드 분석

게임을 구성하는 코드는 [링크](
https://github.com/openai/gym/blob/master/gym/envs/box2d/car_racing.py)에서 확인할 수 있다. 코드는 크게 아래 구조로 구성되어 있다.
- 화면 설정
- Class FrictionDetector
- Class CarRacing
- main

**1. Class FrictionDetector**

두 물체 사이의 접촉 여부를 확인한다. 해당 게임에서는 자동차와 도로간의 접촉 여부로 보상 값과 게임 종료 여부를 판단.
- 만약 방문하지 않은 타일이면 `road_visited`를 True로 바꾸고 보상 1000/N을 추가한다.

**2. Class CarRacing**

게임 환경을 위한 아규먼트 제공
- render_modes : 1.human, 2. rgb_array, 3. state_pixels로 총 3가지 선택지

  하지만 human외의 것을 아규먼트로 전달했을 때는 `pygame.error: video system not initialized` 에러 발생. 아마 코드 내에 render_modes가 human일 때만 수행되는 코드에서 별도로 관련된 초기화를 진행하는 것 같다.
- verbos : 도로에 대한 정보 출력 여부

  [Track generation: {시작_타일_인덱스}, {끝_타일_인덱스} -> {총_타일_수}-tiles track]
- lap_complete_percent : 몇 퍼센트의 타일을 모두 지났을 때 lap을 완료했다고 판단할지를 나타냄.
- domain_randomize : 도로와 배경 색상을 매 시나리오마다 달리할 것인지 여부
- continuous : 액션 종류 정의.

코랩 환경에서는 Box2D Car Racing의 코드 실행 도중 화면을 실시간으로 랜더링하기 어렵다. 환경을 정확히 파악하기 위해서는 화면 랜더링과 사용자의 입력이 동시에 가능해야하므로 별도의 파이썬 코드를 로컬 환경에서 돌리고자 한다.

기존의 코드에서 불필요한 코드는 삭제하고 핵심적인 게임 수행 부분만 실행 가능하도록 수정한다.


다음에는 자율주행을 학습하기 위해 알아야 할 강화학습 기본 개념에 대해 공부한다.